
# Apprentissage par renforcement avec Q-learning

Ce notebook montre comment un agent peut apprendre à se déplacer dans une grille pour atteindre un objectif, en utilisant l'algorithme de Q-learning. Il inclut une visualisation du chemin optimal appris.


In [ ]:

import numpy as np
import random
import matplotlib.pyplot as plt

# Paramètres de la grille
nb_lignes = 4
nb_colonnes = 4
etat_final = (3, 3)

# Actions possibles
actions = ['haut', 'bas', 'gauche', 'droite']
action_index = {a: i for i, a in enumerate(actions)}

# Q-table initialisée à zéro
Q = np.zeros((nb_lignes, nb_colonnes, len(actions)))

# Paramètres Q-learning
alpha = 0.1    # taux d'apprentissage
gamma = 0.9    # facteur de réduction
epsilon = 0.2  # exploration (20%)

# Fonction pour choisir une action (exploration vs exploitation)
def choisir_action(etat):
    if random.uniform(0, 1) < epsilon:
        return random.choice(actions)
    else:
        return actions[np.argmax(Q[etat[0], etat[1]])]

# Fonction pour faire une action et retourner le nouvel état + récompense
def faire_action(etat, action):
    x, y = etat
    if action == 'haut':
        x = max(0, x - 1)
    elif action == 'bas':
        x = min(nb_lignes - 1, x + 1)
    elif action == 'gauche':
        y = max(0, y - 1)
    elif action == 'droite':
        y = min(nb_colonnes - 1, y + 1)

    nouvel_etat = (x, y)
    if nouvel_etat == etat_final:
        return nouvel_etat, 10  # Récompense finale
    else:
        return nouvel_etat, -1  # Pénalité pour mouvement


In [ ]:

# Entraînement Q-learning
nb_episodes = 500

for episode in range(nb_episodes):
    etat = (0, 0)  # Départ
    while etat != etat_final:
        action = choisir_action(etat)
        nouvel_etat, recompense = faire_action(etat, action)
        a_index = action_index[action]

        Q[etat[0], etat[1], a_index] += alpha * (
            recompense + gamma * np.max(Q[nouvel_etat[0], nouvel_etat[1]]) - Q[etat[0], etat[1], a_index]
        )
        etat = nouvel_etat

# Affichage de la Q-table
print("Q-table après entraînement :")
for i in range(nb_lignes):
    for j in range(nb_colonnes):
        print(f"État ({i},{j}) : {Q[i, j]}")


In [ ]:

# Générer le chemin optimal à partir de (0,0)
def chemin_optimal():
    etat = (0, 0)
    chemin = [etat]
    max_etapes = 20  # éviter les boucles infinies
    for _ in range(max_etapes):
        if etat == etat_final:
            break
        action = actions[np.argmax(Q[etat[0], etat[1]])]
        etat, _ = faire_action(etat, action)
        chemin.append(etat)
    return chemin

chemin = chemin_optimal()
print("\nChemin optimal trouvé :")
for e in chemin:
    print(e)


In [ ]:

# Visualisation du chemin
def afficher_grille(chemin):
    grille = np.zeros((nb_lignes, nb_colonnes))
    for idx, (x, y) in enumerate(chemin):
        grille[x, y] = idx + 1  # marquer les étapes

    plt.figure(figsize=(6,6))
    plt.imshow(grille, cmap='Blues', origin='upper')

    for i in range(nb_lignes):
        for j in range(nb_colonnes):
            txt = 'G' if (i,j)==etat_final else int(grille[i,j]) if grille[i,j]>0 else ''
            plt.text(j, i, txt, ha='center', va='center', fontsize=12, color='black')

    plt.title("Chemin optimal appris")
    plt.xticks(np.arange(nb_colonnes))
    plt.yticks(np.arange(nb_lignes))
    plt.grid(True)
    plt.show()

afficher_grille(chemin)
